In [1]:
import numpy as np
import math
import cv2
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import sklearn

In [2]:
### Read the csv
import os
import csv

sim_data_set = []
csv_path = './data/driving_log.csv'
with open(csv_path) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        sim_data_set.append(line)
#From csv file, first line is header, so remove header line
sim_data_set = sim_data_set[1:]
print('Data import done!')

Data import done!


In [3]:
###Define functions for image pre-proceesing(crop and resize) and fake data formation so that they can be called 
###in generator function

def pre_process(img):
    #Crop image to see section of the road
    crop_image = img[50:140, :]
    #Resize
    return cv2.resize(crop_image,(66,200)) 

def left_images(data):
    source_path=data[1]
    img_name=source_path.split('/')[-1]
    current_path='./data/IMG/'+img_name
    image=cv2.imread(current_path)
    image=pre_process(image)
    measurement=float(data[3])+0.25 #correction factor
    return image,measurement

def right_images(data):
    source_path=data[2]
    img_name=source_path.split('/')[-1]
    current_path='./data/IMG/'+img_name
    image=cv2.imread(current_path)
    image=pre_process(image)
    measurement=float(data[3])-0.25 #correction factor
    return image,measurement

def flip_image(image):
    return cv2.flip(image, 1)

def change_brightness(image):
    dst = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    dst[:,:,2] = dst[:,:,2]*np.random.uniform(low = 0.4, high = 1.0)
    return cv2.cvtColor(dst, cv2.COLOR_HSV2RGB)
    
# Shifts the images horizontally and vertically
def shift_image(image, center_angle):
    x_shift = np.random.randint(low = -20, high = 20)
    y_shift = np.random.randint(low = -8, high = 8)
    shift = np.float32([[1,0, x_shift],[0,1,y_shift]])
    corrected_angle = np.sign(x_shift)*0.009 + center_angle
    return cv2.warpAffine(image,shift,(66,200)), corrected_angle

In [4]:
#Define generator function to create data for each epoch
#Sample is whole line in original dataset, we need to read inages and process them

def generator(samples):
    batch_size=32
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            #Initilize dataset
            images = []
            angles = []
            for batch_sample in batch_samples:
                #Read center image and angle
                name = './data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_image=pre_process(center_image)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                #if angle is not zero, I would flip the image.
                if center_angle != 0:
                    #Include flip image
                    flipped_img=flip_image(center_image)
                    flipped_angle=-center_angle
                    images.append(center_image)
                    angles.append(center_angle)
                
                #Include image with change in brightness
                bright_img=change_brightness(center_image)
                images.append(bright_img)
                angles.append(center_angle)
                    
                #Include shifted image
                shift_img,shift_angle=shift_image(center_image,center_angle)
                images.append(shift_img)
                angles.append(shift_angle)
                
                #include left image
                left_img,left_angle=left_images(batch_sample)
                images.append(left_img)
                angles.append(left_angle)
                
                #include right image
                right_img,right_angle=right_images(batch_sample)
                images.append(right_img)
                angles.append(right_angle)
            
            #Keras needs input in the form of arrays
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)


In [5]:
### Model architecture: NVDIA
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Lambda(lambda x: (x / 255) - 0.5, output_shape = (200,66,3), input_shape = (200,66,3))) 
model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(50))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(10))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(1))

Using TensorFlow backend.


In [6]:
### Configures the learning process
model.compile(optimizer='adam',loss='mean_squared_error')

### Split into train samples and validation samples
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(sim_data_set, test_size=0.2)

### Train the model
train_generator = generator(train_samples) 
validation_generator = generator(validation_samples)
        
history = model.fit_generator(train_generator,samples_per_epoch= 58000, 
        validation_data=validation_generator, nb_val_samples=len(validation_samples),nb_epoch=5)

### Save the model
model.save('./model.h5')  # creates a HDF5 file

print('Model training done!')

Epoch 1/5
57926/58000 [============================>.] - ETA: 0s - loss: 0.0364

/Users/noopur/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


58103/58000 [==============================] - 627s - loss: 0.0364 - val_loss: 0.0310
Epoch 2/5
58073/58000 [==============================] - 612s - loss: 0.0279 - val_loss: 0.0267
Epoch 3/5
58106/58000 [==============================] - 612s - loss: 0.0257 - val_loss: 0.0209
Epoch 4/5
58064/58000 [==============================] - 612s - loss: 0.0247 - val_loss: 0.0303
Epoch 5/5
58092/58000 [==============================] - 622s - loss: 0.0240 - val_loss: 0.0218
Model training done!
